In [3]:
import json
import uuid
import os

def convert_coco_to_via_with_names(coco_json_path, output_path):
    # Load COCO JSON
    with open(coco_json_path, 'r') as f:
        coco = json.load(f)

    # Map image ID to metadata
    image_id_to_info = {img['id']: img for img in coco['images']}

    # Map category ID to name
    category_id_to_name = {cat['id']: cat['name'] for cat in coco['categories']}

    # Group annotations by image_id
    image_id_to_annotations = {}
    for ann in coco['annotations']:
        if ann.get('iscrowd', 0):
            continue  # skip crowd annotations
        image_id_to_annotations.setdefault(ann['image_id'], []).append(ann)

    # Build VIA-format dictionary
    via = {}
    for image_id, anns in image_id_to_annotations.items():
        image_info = image_id_to_info[image_id]
        filename = image_info['file_name']
        regions = []

        for ann in anns:
            if isinstance(ann['segmentation'], list):  # Only polygon format
                for seg in ann['segmentation']:
                    if len(seg) < 6:
                        continue  # Invalid polygon
                    all_points_x = seg[::2]
                    all_points_y = seg[1::2]
                    region = {
                        "shape_attributes": {
                            "name": "polygon",
                            "all_points_x": all_points_x,
                            "all_points_y": all_points_y
                        },
                        "region_attributes": {
                            "names": category_id_to_name.get(ann['category_id'], "unknown")
                        }
                    }
                    regions.append(region)

        key = str(uuid.uuid4())[:16]  # Unique ID for VIA
        via[key] = {
            "filename": filename,
            "size": 0,  # Optional
            "regions": regions,
            "file_attributes": {}
        }

    # Save output JSON
    with open(output_path, 'w') as f:
        json.dump(via, f, indent=2)

    print(f"✅ VIA-format JSON with 'names' saved to: {output_path}")


# ----------- USAGE EXAMPLE -----------

if __name__ == "__main__":
    # Set your paths here
    coco_input_path = r"C:\Users\My PC\Desktop\maskrcnn\no-augment\val\_annotations.coco.json"
    via_output_path = r"C:\Users\My PC\Desktop\maskrcnn\no-augment\val\via_converted_annotations_with_names.json"

    # Run the converter
    convert_coco_to_via_with_names(coco_input_path, via_output_path)


✅ VIA-format JSON with 'names' saved to: C:\Users\My PC\Desktop\maskrcnn\no-augment\val\via_converted_annotations_with_names.json


In [ ]:
# import json
# import os
# from collections import defaultdict

# def coco_to_via(coco_json_paths, output_path):
#     via_data = {}
#     ann_id_map = defaultdict(list)

#     for coco_json_path in coco_json_paths:
#         with open(coco_json_path, 'r') as f:
#             coco = json.load(f)

#         images = {img['id']: img for img in coco['images']}
#         categories = {cat['id']: cat['name'] for cat in coco['categories']}

#         for ann in coco['annotations']:
#             image_id = ann['image_id']
#             img_info = images[image_id]
#             file_name = img_info['file_name']
#             file_key = file_name  # VIA uses filename as key

#             if file_key not in via_data:
#                 via_data[file_key] = {
#                     'filename': file_name,
#                     'size': -1,
#                     'regions': [],
#                     'file_attributes': {}
#                 }

#             if ann.get("iscrowd", 0) == 1:
#                 continue  # skip RLE encoded masks for simplicity

#             segmentation = ann['segmentation'][0]  # flatten polygon
#             all_points_x = segmentation[::2]
#             all_points_y = segmentation[1::2]
#             category_name = categories[ann['category_id']]

#             region = {
#                 "shape_attributes": {
#                     "name": "polygon",
#                     "all_points_x": all_points_x,
#                     "all_points_y": all_points_y
#                 },
#                 "region_attributes": {
#                     "names": category_name  # Rename here if needed
#                 }
#             }

#             via_data[file_key]['regions'].append(region)

#     # Convert dict to VIA-style with unique keys
#     via_final = {}
#     for idx, (key, data) in enumerate(via_data.items()):
#         via_key = f"{key}{idx}"
#         via_final[via_key] = data

#     # Save the combined VIA annotation
#     with open(output_path, 'w') as f:
#         json.dump(via_final, f, indent=2)

#     print(f"✅ VIA annotations saved to: {output_path}")


# # 🔧 Example usage
# coco_json_paths = [
#     r"C:\Users\My PC\Desktop\maskrcnn\no-augment\train\_annotations.coco.json",
#     r"C:\Users\My PC\Desktop\maskrcnn\no-augment\valid\_annotations.coco.json",
#     r"C:\Users\My PC\Desktop\maskrcnn\no-augment\test\_annotations.coco.json"
# ]

# output_path = r"C:\Users\My PC\Desktop\maskrcnn\no-augment\via_converted_annotations_with_names.json"

# coco_to_via(coco_json_paths, output_path)


✅ VIA annotations saved to: C:\Users\My PC\Desktop\maskrcnn\no-augment\via_converted_annotations_with_names.json


In [10]:
!nvidia-smi

Fri Aug  8 17:36:00 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 576.57                 Driver Version: 576.57         CUDA Version: 12.9     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                  Driver-Model | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA GeForce RTX 4050 ...  WDDM  |   00000000:01:00.0 Off |                  N/A |
| N/A   44C    P8              1W /   80W |      79MiB /   6141MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [2]:
import tensorflow as tf
print("Num GPUs Available:", len(tf.config.experimental.list_physical_devices('GPU')))


Num GPUs Available: 0


In [3]:
import numpy as np
print(np.__version__)


1.19.5
